
# 🏞️ Random-Forest Autoencoder (RFAE) — Colab Notebook

This notebook demonstrates training, encoding, decoding, and evaluation for **Autoencoding Random Forests (RFAE)** in Python using **PyTorch + scikit-learn**.

**References**
- *Autoencoding Random Forests* — Binh Duc Vu, Jan Kapar, Marvin N. Wright, David S. Watson (arXiv:2505.21441, 2025).
- Official research repo (R): https://github.com/bips-hb/RFAE


In [ ]:

#@title 🔧 Setup: install dependencies
!pip -q install numpy pandas scikit-learn torch scipy matplotlib
print("✅ Dependencies installed")


In [ ]:

#@title 📦 Create the local module `rfae.py` (auto-generated)
%%writefile rfae.py
# rfae.py
# MIT License (c) 2025
"""
Random-Forest Autoencoder (RFAE) — PyTorch + scikit-learn implementation

Encoder: Diffusion-map embedding on a Random-Forest (RF) kernel with Nyström out-of-sample.
Decoder: k-NN in latent space (weighted mean for numeric; weighted vote for categoricals).
Optional: exact-but-slower synthetic representatives via leaf-box intersections.
Optional: tiny neural encoder (Torch MLP) to approximate Nyström for high-throughput inference.

References
---------
- Binh Duc Vu, Jan Kapar, Marvin N. Wright, David S. Watson. "Autoencoding Random Forests."
  arXiv:2505.21441 (2025).
- Official R/Python reference repo: https://github.com/bips-hb/RFAE
"""

from __future__ import annotations

import dataclasses
import json
import math
import warnings
from dataclasses import dataclass
from typing import Any, Dict, Iterable, List, Optional, Tuple, Union

import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import (
    RandomForestClassifier,
    RandomForestRegressor,
    RandomTreesEmbedding,
)
from sklearn.utils.validation import check_is_fitted
from sklearn.utils import check_random_state

try:
    import torch
    TORCH_AVAILABLE = True
except Exception:
    TORCH_AVAILABLE = False

try:
    from scipy.sparse.linalg import eigsh  # type: ignore
    SCIPY_AVAILABLE = True
except Exception:
    SCIPY_AVAILABLE = False


@dataclass
class RFAEConfig:
    # Forest
    n_estimators: int = 500
    max_depth: Optional[int] = None
    min_samples_leaf: int = 1
    random_state: Optional[int] = 42
    n_jobs: Optional[int] = None

    # Mode: "supervised-classification", "supervised-regression", "unsupervised"
    mode: str = "supervised-classification"

    # Diffusion map
    latent_dim: int = 16
    diffusion_time: float = 1.0
    use_torch: bool = True
    device: Optional[str] = None

    # Decoder (k-NN)
    k_neighbors: int = 50
    distance_eps: float = 1e-8
    weight_power: float = 2.0
    exact_decoder: bool = False
    max_trees_for_intersection: Optional[int] = 256

    # Preprocessing
    standardize_numeric: bool = True
    categorical_strategy: str = "ordinal"
    handle_unknown_cats: str = "use_encoded_value"

    # Numerics
    eigen_solver: str = "auto"  # "auto" | "numpy" | "scipy" | "torch"
    float_dtype: str = "float64"

    verbose: bool = True


def _infer_columns(X: Union[pd.DataFrame, np.ndarray]) -> Tuple[List[str], List[str]]:
    if isinstance(X, pd.DataFrame):
        cat_cols = [c for c in X.columns
                    if pd.api.types.is_object_dtype(X[c]) or pd.api.types.is_categorical_dtype(X[c])]
        num_cols = [c for c in X.columns if c not in cat_cols]
        return num_cols, cat_cols
    else:
        n = X.shape[1]
        return [f"x{i}" for i in range(n)], []


def _make_preprocessor(X: Union[pd.DataFrame, np.ndarray], cfg: RFAEConfig) -> Tuple[Pipeline, List[str], List[str]]:
    num_cols, cat_cols = _infer_columns(X)
    transformers = []
    if len(num_cols) > 0:
        transformers.append(("num", StandardScaler() if cfg.standardize_numeric else "passthrough", num_cols))
    if len(cat_cols) > 0:
        if cfg.categorical_strategy == "ordinal":
            oe = OrdinalEncoder(handle_unknown=cfg.handle_unknown_cats, unknown_value=-1)
            transformers.append(("cat", oe, cat_cols))
        else:
            ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
            transformers.append(("cat", ohe, cat_cols))
    pre = ColumnTransformer(transformers, remainder="drop")
    return Pipeline([("pre", pre)]), num_cols, cat_cols


class RFAE(BaseEstimator, TransformerMixin):
    def __init__(self, config: Optional[RFAEConfig] = None):
        self.cfg = config or RFAEConfig()
        self._rng = check_random_state(self.cfg.random_state)

        self.preprocessor_: Optional[Pipeline] = None
        self.num_cols_: List[str] = []
        self.cat_cols_: List[str] = []
        self.is_supervised_: bool = False
        self.task_: str = "classification"

        self.forest_: Any = None
        self.estimators_: List[Any] = []

        self.K_: Optional[np.ndarray] = None
        self.evals_: Optional[np.ndarray] = None
        self.evecs_: Optional[np.ndarray] = None
        self.Z_train_: Optional[np.ndarray] = None
        self.Z_scaler_: float = 1.0

        self.X_train_pre_: Optional[np.ndarray] = None
        self.X_train_df_: Optional[pd.DataFrame] = None
        self.n_samples_: int = 0
        self.n_features_: int = 0

        self._leaf_index_per_tree_: List[Dict[int, np.ndarray]] = []
        self._leaf_size_per_tree_: List[Dict[int, int]] = []

        self._synthetic_train_X_: Optional[np.ndarray] = None

    def fit(self, X: Union[pd.DataFrame, np.ndarray], y: Optional[Union[np.ndarray, Iterable]] = None) -> "RFAE":
        X_df = X if isinstance(X, pd.DataFrame) else pd.DataFrame(X, columns=[f"x{i}" for i in range(np.asarray(X).shape[1])])
        self.X_train_df_ = X_df.copy()

        self.preprocessor_, self.num_cols_, self.cat_cols_ = _make_preprocessor(X_df, self.cfg)
        Xp = np.asarray(self.preprocessor_.fit_transform(X_df), dtype=self.cfg.float_dtype)
        self.X_train_pre_ = Xp
        self.n_samples_, self.n_features_ = Xp.shape

        self.is_supervised_ = self.cfg.mode.startswith("supervised")
        if self.is_supervised_:
            if y is None:
                raise ValueError("Supervised mode selected but y is None.")
            y = np.asarray(y)
            if self.cfg.mode == "supervised-classification":
                self.task_ = "classification"
                self.forest_ = RandomForestClassifier(
                    n_estimators=self.cfg.n_estimators,
                    max_depth=self.cfg.max_depth,
                    min_samples_leaf=self.cfg.min_samples_leaf,
                    n_jobs=self.cfg.n_jobs,
                    random_state=self.cfg.random_state,
                )
            elif self.cfg.mode == "supervised-regression":
                self.task_ = "regression"
                self.forest_ = RandomForestRegressor(
                    n_estimators=self.cfg.n_estimators,
                    max_depth=self.cfg.max_depth,
                    min_samples_leaf=self.cfg.min_samples_leaf,
                    n_jobs=self.cfg.n_jobs,
                    random_state=self.cfg.random_state,
                )
            else:
                raise ValueError(f"Unknown supervised mode: {self.cfg.mode}")
            self.forest_.fit(Xp, y)
            self.estimators_ = list(self.forest_.estimators_)
        else:
            self.task_ = "unsupervised"
            rte = RandomTreesEmbedding(
                n_estimators=self.cfg.n_estimators,
                max_depth=self.cfg.max_depth,
                n_jobs=self.cfg.n_jobs,
                random_state=self.cfg.random_state,
            )
            rte.fit(Xp)
            self.forest_ = rte
            self.estimators_ = list(rte.estimators_)

        if self.cfg.verbose:
            print(f"[RFAE] Fitted forest with {len(self.estimators_)} trees; building RF kernel...")

        self._leaf_index_per_tree_.clear()
        self._leaf_size_per_tree_.clear()
        for est in self.estimators_:
            leaves = est.apply(self.X_train_pre_)
            leaf_to_idx = {int(lid): np.where(leaves == lid)[0] for lid in np.unique(leaves)}
            self._leaf_index_per_tree_.append(leaf_to_idx)
            self._leaf_size_per_tree_.append({lid: len(idx) for lid, idx in leaf_to_idx.items()})

        self.K_ = self._build_rf_kernel_train()
        if self.cfg.verbose:
            print("[RFAE] RF kernel built. Row sum (first 5):", np.round(self.K_.sum(1)[:5], 3))

        self.evals_, self.evecs_ = self._eigendecompose(self.K_, top_k=self.cfg.latent_dim + 1)
        evals_nt = self.evals_[1: self.cfg.latent_dim + 1]
        evecs_nt = self.evecs_[:, 1: self.cfg.latent_dim + 1]
        self.Z_scaler_ = float(np.sqrt(self.n_samples_))
        self.Z_train_ = (self.Z_scaler_ * evecs_nt) * (evals_nt ** self.cfg.diffusion_time)

        if self.cfg.exact_decoder:
            if self.cfg.verbose:
                print("[RFAE] Precomputing synthetic representatives (exact decoder)...")
            self._synthetic_train_X_ = self._build_synthetic_training_points()

        return self

    def encode(self, X: Union[pd.DataFrame, np.ndarray]) -> np.ndarray:
        check_is_fitted(self, ["Z_train_", "evals_", "evecs_", "K_"])
        X_df = X if isinstance(X, pd.DataFrame) else pd.DataFrame(X, columns=self.X_train_df_.columns)
        Xp = np.asarray(self.preprocessor_.transform(X_df), dtype=self.cfg.float_dtype)

        K0 = self._build_rf_kernel_test(Xp)
        evals_nt = self.evals_[1: self.cfg.latent_dim + 1]
        Z0 = K0 @ (self.Z_train_ * (evals_nt ** -1.0))
        return np.asarray(Z0, dtype=self.cfg.float_dtype)

    def decode(self, Z: np.ndarray, k: Optional[int] = None) -> Union[np.ndarray, pd.DataFrame]:
        check_is_fitted(self, ["Z_train_", "X_train_pre_"])
        k = k or self.cfg.k_neighbors
        Z = np.asarray(Z, dtype=self.cfg.float_dtype)

        from sklearn.neighbors import NearestNeighbors
        nbrs = NearestNeighbors(n_neighbors=min(k, len(self.Z_train_))).fit(self.Z_train_)
        dists, inds = nbrs.kneighbors(Z, return_distance=True)

        w = 1.0 / np.maximum(dists, self.cfg.distance_eps) ** self.cfg.weight_power
        w = (w / w.sum(axis=1, keepdims=True)).astype(self.cfg.float_dtype)

        Xrep = self._synthetic_train_X_ if (self.cfg.exact_decoder and self._synthetic_train_X_ is not None) else self.X_train_pre_
        X_rep_neighbors = Xrep[inds]
        X_cont_recon = np.einsum("mk,mkd->md", w, X_rep_neighbors)

        out_df = pd.DataFrame(index=range(Z.shape[0]))
        if len(self.num_cols_) > 0:
            n_num = len(self.num_cols_)
            out_df[self.num_cols_] = X_cont_recon[:, :n_num]

        if len(self.cat_cols_) > 0:
            X_train_raw = self.X_train_df_.reset_index(drop=True)
            cat_vals_neighbors = X_train_raw.loc[inds.flatten(), self.cat_cols_].to_numpy().reshape(Z.shape[0], -1, len(self.cat_cols_))
            for j, col in enumerate(self.cat_cols_):
                winners = []
                for i in range(Z.shape[0]):
                    label_to_w = {}
                    for lbl, ww in zip(cat_vals_neighbors[i, :, j], w[i]):
                        label_to_w[lbl] = label_to_w.get(lbl, 0.0) + float(ww)
                    top = max(label_to_w.values())
                    cands = [lbl for lbl, ww in label_to_w.items() if abs(ww - top) < 1e-12]
                    winners.append(self._rng.choice(cands))
                out_df[col] = np.array(winners, dtype=object)
        return out_df

    def _build_rf_kernel_train(self) -> np.ndarray:
        n = self.n_samples_
        K = np.zeros((n, n), dtype=self.cfg.float_dtype)
        B = float(len(self.estimators_))
        for leaf_to_idx in self._leaf_index_per_tree_:
            for lid, idx in leaf_to_idx.items():
                sz = len(idx)
                if sz > 0:
                    K[np.ix_(idx, idx)] += (1.0 / sz)
        K /= B
        K = 0.5 * (K + K.T)
        K[K < 0] = 0.0
        return K

    def _build_rf_kernel_test(self, X_test_pre: np.ndarray) -> np.ndarray:
        m, n = X_test_pre.shape[0], self.n_samples_
        K0 = np.zeros((m, n), dtype=self.cfg.float_dtype)
        B = float(len(self.estimators_))
        for est, leaf_to_idx, leaf_sizes in zip(self.estimators_, self._leaf_index_per_tree_, self._leaf_size_per_tree_):
            test_leaves = est.apply(X_test_pre)
            for i, lid in enumerate(test_leaves):
                idx = leaf_to_idx.get(int(lid))
                if idx is None:
                    continue
                sz = leaf_sizes[int(lid)]
                if sz > 0:
                    K0[i, idx] += (1.0 / sz)
        K0 /= B
        row_sums = K0.sum(axis=1, keepdims=True)
        row_sums[row_sums == 0] = 1.0
        K0 /= row_sums
        return K0

    def _eigendecompose(self, K: np.ndarray, top_k: int) -> Tuple[np.ndarray, np.ndarray]:
        solver = self.cfg.eigen_solver
        if solver == "auto":
            if TORCH_AVAILABLE and self.cfg.use_torch:
                solver = "torch"
            elif SCIPY_AVAILABLE and K.shape[0] >= 512:
                solver = "scipy"
            else:
                solver = "numpy"

        if solver == "torch" and TORCH_AVAILABLE:
            device = self.cfg.device or ("cuda" if torch.cuda.is_available() else "cpu")
            tK = torch.tensor(K, dtype=torch.float64 if self.cfg.float_dtype == "float64" else torch.float32, device=device)
            evals, evecs = torch.linalg.eigh(tK)
            evals = evals.detach().cpu().numpy()
            evecs = evecs.detach().cpu().numpy()
        elif solver == "scipy" and SCIPY_AVAILABLE:
            k = min(top_k, K.shape[0] - 1)
            evals, evecs = eigsh(K, k=k, which="LA")
        else:
            evals, evecs = np.linalg.eigh(K)

        idx = np.argsort(evals)[::-1]
        return evals[idx], evecs[:, idx]

    def _build_synthetic_training_points(self) -> np.ndarray:
        Xp = self.X_train_pre_
        n, d = Xp.shape
        global_min = np.min(Xp, axis=0)
        global_max = np.max(Xp, axis=0)
        trees = self.estimators_[: self.cfg.max_trees_for_intersection] if self.cfg.max_trees_for_intersection else self.estimators_
        lows = np.tile(global_min, (n, 1)).astype(self.cfg.float_dtype)
        highs = np.tile(global_max, (n, 1)).astype(self.cfg.float_dtype)

        for est in trees:
            tree = est.tree_
            path = est.decision_path(Xp)
            features = tree.feature
            thresholds = tree.threshold
            for i in range(n):
                node_indices = path.indices[path.indptr[i]: path.indptr[i+1]]
                for node in node_indices:
                    feat = features[node]
                    if feat < 0:
                        continue
                    thr = thresholds[node]
                    if Xp[i, feat] <= thr:
                        highs[i, feat] = min(highs[i, feat], thr)
                    else:
                        import numpy as _np
                        lows[i, feat] = max(lows[i, feat], _np.nextafter(thr, _np.inf))

        lows = np.minimum(lows, highs)
        widths = np.maximum(highs - lows, 0.0)
        U = self._rng.uniform(size=(n, d)).astype(self.cfg.float_dtype)
        return lows + U * widths

    def save(self, path: Union[str, 'Path']) -> None:
        from pathlib import Path as _Path
        check_is_fitted(self, ["Z_train_", "evecs_", "evals_"])
        path = _Path(path)
        path.parent.mkdir(parents=True, exist_ok=True)
        meta = {
            "cfg": dataclasses.asdict(self.cfg),
            "num_cols": self.num_cols_,
            "cat_cols": self.cat_cols_,
            "is_supervised": self.is_supervised_,
            "task": self.task_,
            "n_samples": self.n_samples_,
            "n_features": self.n_features_,
            "Z_scaler": self.Z_scaler_,
        }
        with open(path.with_suffix(".json"), "w") as f:
            json.dump(meta, f, indent=2)
        import numpy as _np
        _np.savez_compressed(path.with_suffix(".npz"),
                             K=self.K_, evals=self.evals_, evecs=self.evecs_,
                             Z_train=self.Z_train_, X_train_pre=self.X_train_pre_)
        try:
            import joblib
            joblib.dump({"preprocessor": self.preprocessor_, "forest": self.forest_, "estimators": self.estimators_},
                        path.with_suffix(".skjoblib"))
        except Exception as e:
            warnings.warn(f"Failed to save sklearn objects with joblib: {e}")

    @staticmethod
    def load(path: Union[str, 'Path']) -> "RFAE":
        from pathlib import Path as _Path
        path = _Path(path)
        with open(path.with_suffix(".json"), "r") as f:
            meta = json.load(f)
        cfg = RFAEConfig(**meta["cfg"])
        model = RFAE(cfg)
        model.num_cols_ = meta["num_cols"]
        model.cat_cols_ = meta["cat_cols"]
        model.is_supervised_ = meta["is_supervised"]
        model.task_ = meta["task"]
        model.n_samples_ = meta["n_samples"]
        model.n_features_ = meta["n_features"]
        model.Z_scaler_ = meta["Z_scaler"]
        blob = np.load(path.with_suffix(".npz"))
        model.K_ = blob["K"]; model.evals_ = blob["evals"]; model.evecs_ = blob["evecs"]
        model.Z_train_ = blob["Z_train"]; model.X_train_pre_ = blob["X_train_pre"]
        try:
            import joblib
            objs = joblib.load(path.with_suffix(".skjoblib"))
            model.preprocessor_ = objs["preprocessor"]
            model.forest_ = objs["forest"]
            model.estimators_ = objs.get("estimators") or getattr(model.forest_, "estimators_", [])
            model._leaf_index_per_tree_.clear(); model._leaf_size_per_tree_.clear()
            leaves_all = [est.apply(model.X_train_pre_) for est in model.estimators_]
            for leaves in leaves_all:
                leaf_to_idx = {int(lid): np.where(leaves == lid)[0] for lid in np.unique(leaves)}
                model._leaf_index_per_tree_.append(leaf_to_idx)
                model._leaf_size_per_tree_.append({lid: len(idx) for lid, idx in leaf_to_idx.items()})
        except Exception as e:
            warnings.warn(f"Failed to load sklearn objects with joblib: {e}")
        return model


# ---- Evaluation helper ----
def reconstruction_distortion(
    X_true: Union[pd.DataFrame, np.ndarray],
    X_recon: Union[pd.DataFrame, np.ndarray],
) -> Dict[str, Any]:
    if isinstance(X_true, np.ndarray):
        X_true_df = pd.DataFrame(X_true, columns=[f"x{i}" for i in range(X_true.shape[1])])
    else:
        X_true_df = X_true.copy()
    if isinstance(X_recon, np.ndarray):
        X_recon_df = pd.DataFrame(X_recon, columns=X_true_df.columns)
    else:
        X_recon_df = X_recon.copy()
    num_cols = [c for c in X_true_df.columns if pd.api.types.is_numeric_dtype(X_true_df[c])]
    cat_cols = [c for c in X_true_df.columns if c not in num_cols]

    cont_scores = []
    for c in num_cols:
        x = X_true_df[c].to_numpy(dtype=float)
        y = X_recon_df[c].to_numpy(dtype=float)
        var = np.var(x)
        if var < 1e-12: continue
        mse = np.mean((x - y) ** 2)
        cont_scores.append(mse / var)
    cont_mean = float(np.mean(cont_scores)) if cont_scores else np.nan

    cat_scores = []
    for c in cat_cols:
        x = X_true_df[c].astype(str).to_numpy()
        y = X_recon_df[c].astype(str).to_numpy()
        cat_scores.append(float(np.mean(x != y)))
    cat_mean = float(np.mean(cat_scores)) if cat_scores else np.nan

    parts = [v for v in [cont_mean, cat_mean] if not np.isnan(v)]
    overall = float(np.mean(parts)) if parts else np.nan
    return {"continuous": cont_mean, "categorical": cat_mean, "overall": overall}


# ---- Optional: neural encoder (Torch) ----
if TORCH_AVAILABLE:
    class _MLP(torch.nn.Module):
        def __init__(self, in_dim: int, out_dim: int, hidden: List[int]):
            super().__init__()
            layers = []
            last = in_dim
            for h in hidden:
                layers += [torch.nn.Linear(last, h), torch.nn.ReLU()]
                last = h
            layers += [torch.nn.Linear(last, out_dim)]
            self.net = torch.nn.Sequential(*layers)
        def forward(self, x):
            return self.net(x)

    def _to_tensor(x: np.ndarray, device: str, dtype):
        return torch.tensor(x, device=device, dtype=dtype)

    def _from_tensor(x: "torch.Tensor") -> np.ndarray:
        return x.detach().cpu().numpy()

    def fit_neural_encoder(self: "RFAE",
                           hidden: List[int] = [256, 128],
                           epochs: int = 50,
                           batch_size: int = 256,
                           lr: float = 1e-3,
                           weight_decay: float = 1e-6,
                           patience: int = 10,
                           verbose: bool = True) -> "RFAE":
        check_is_fitted(self, ["Z_train_", "X_train_pre_"])
        device = self.cfg.device or ("cuda" if torch.cuda.is_available() else "cpu")
        dtype = torch.float32 if self.cfg.float_dtype == "float32" else torch.float64
        model = _MLP(self.n_features_, self.cfg.latent_dim, hidden).to(device=device, dtype=dtype)
        opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        loss_fn = torch.nn.MSELoss()

        X = _to_tensor(self.X_train_pre_, device, dtype)
        Z = _to_tensor(self.Z_train_, device, dtype)

        n = X.shape[0]
        idx = np.arange(n)
        loss_hist: List[float] = []

        def early_stop(hist, p):
            if len(hist) < p + 1: return False
            best = min(hist[:-p])
            return all(l >= best for l in hist[-p:])

        for ep in range(1, epochs + 1):
            self._rng.shuffle(idx)
            model.train(); ep_loss = 0.0
            for s in range(0, n, batch_size):
                b = idx[s:s+batch_size]
                pred = model(X[b]); loss = loss_fn(pred, Z[b])
                opt.zero_grad(set_to_none=True); loss.backward(); opt.step()
                ep_loss += float(loss.detach().cpu().item()) * len(b)
            ep_loss /= float(n)
            loss_hist.append(ep_loss)
            if verbose: print(f"[neural-encoder] epoch {ep:03d}/{epochs}  loss={ep_loss:.6f}")
            if early_stop(loss_hist, patience):
                if verbose: print("[neural-encoder] early stopping")
                break

        self._nn_encoder = model
        self._nn_device = device
        self._nn_dtype = dtype
        return self

    def encode_nn(self: "RFAE", X: Union[pd.DataFrame, np.ndarray]) -> np.ndarray:
        if not hasattr(self, "_nn_encoder"):
            raise RuntimeError("Neural encoder not trained. Call fit_neural_encoder first.")
        X_df = X if isinstance(X, pd.DataFrame) else pd.DataFrame(X, columns=self.X_train_df_.columns)
        Xp = np.asarray(self.preprocessor_.transform(X_df), dtype=self.cfg.float_dtype)
        self._nn_encoder.eval()
        with torch.no_grad():
            z = self._nn_encoder(_to_tensor(Xp, self._nn_device, self._nn_dtype))
        return _from_tensor(z)

    setattr(RFAE, "fit_neural_encoder", fit_neural_encoder)
    setattr(RFAE, "encode_nn", encode_nn)


def _demo_iris():
    from sklearn.datasets import load_iris
    data = load_iris(as_frame=True)
    X = data.frame.drop(columns=["target"]); y = data.frame["target"]
    cfg = RFAEConfig(mode="supervised-classification", n_estimators=300, latent_dim=4, use_torch=False, verbose=True)
    rfae = RFAE(cfg).fit(X, y)
    Z = rfae.encode(X.iloc[:5])
    Xh = rfae.decode(Z)
    print("Z shape:", Z.shape); print(Xh.head())

if __name__ == "__main__":
    _demo_iris()



In [ ]:

#@title 📚 Import the module
from rfae import RFAE, RFAEConfig, reconstruction_distortion
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.datasets import load_iris, make_moons
from sklearn.model_selection import train_test_split
from time import time
print("✅ rfae.py imported")


In [ ]:

#@title 📈 Helper: scatter plot for embeddings
import numpy as np, matplotlib.pyplot as plt
def plot_embedding(Z, labels=None, title="Diffusion Embedding (RFAE)"):
    Z = np.asarray(Z)
    if Z.shape[1] < 2: raise ValueError("Need at least 2D latent space to plot.")
    plt.figure(figsize=(6,5))
    if labels is None:
        plt.scatter(Z[:,0], Z[:,1], s=16, alpha=0.8)
    else:
        labels = np.asarray(labels)
        for u in np.unique(labels):
            m = (labels == u)
            plt.scatter(Z[m,0], Z[m,1], s=16, alpha=0.8, label=str(u))
        plt.legend(loc="best", fontsize=10)
    plt.title(title); plt.xlabel("Z1"); plt.ylabel("Z2"); plt.show()


## 🌸 Supervised demo (Iris)

In [ ]:

data = load_iris(as_frame=True)
X = data.frame.drop(columns=["target"]); y = data.frame["target"]
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

cfg = RFAEConfig(mode="supervised-classification", n_estimators=500, latent_dim=8, diffusion_time=1.0, k_neighbors=40, use_torch=True, verbose=True)

t0 = time(); model = RFAE(cfg).fit(Xtr, ytr); fit_time = time()-t0
Ztr = model.encode(Xtr); Zte = model.encode(Xte)
Xhat = model.decode(Zte); dist = reconstruction_distortion(Xte, Xhat)

print(f"✅ Fit time: {fit_time:.2f}s")
print("📊 Distortion (lower is better):", dist)
plot_embedding(np.vstack([Ztr[:,:2], Zte[:,:2]]), labels=np.hstack([ytr.values, yte.values]), title="Iris — RFAE diffusion embedding (2D)")


### 💾 Save & reload

In [ ]:

model.save("rfae_iris")
reloaded = RFAE.load("rfae_iris")
Zte2 = reloaded.encode(Xte)
print("Δ encoding L2 between pre/post load:", float(np.linalg.norm(Zte - Zte2)))


## 🌙 Unsupervised demo (Two Moons)

In [ ]:

from sklearn.datasets import make_moons
X_moons, y_moons = make_moons(n_samples=1500, noise=0.07, random_state=42)
import pandas as pd
X_moons = pd.DataFrame(X_moons, columns=["x1","x2"])

cfg_unsup = RFAEConfig(mode="unsupervised", n_estimators=800, latent_dim=8, k_neighbors=50, use_torch=True, verbose=False)
unsup = RFAE(cfg_unsup).fit(X_moons)
Z_moons = unsup.encode(X_moons)
plot_embedding(Z_moons[:, :2], labels=y_moons, title="Unsupervised RFAE on Two Moons")
Xhat_moons = unsup.decode(Z_moons)
Xhat_moons.head()


## ⚡ Optional: neural encoder to approximate Nyström

In [ ]:

model.fit_neural_encoder(hidden=[256,128], epochs=40, lr=1e-3, patience=6, verbose=True)
from time import time
t0 = time(); Z_fast = model.encode_nn(Xte); t_fast = time()-t0
t0 = time(); Z_ref  = model.encode(Xte);  t_ref  = time()-t0
approx_err = float(np.linalg.norm(Z_fast - Z_ref) / (np.linalg.norm(Z_ref)+1e-8))
print(f"⏱️ encode_nn: {t_fast*1000:.1f} ms vs Nyström: {t_ref*1000:.1f} ms; relative error ~ {approx_err:.4f}")



---
### Notes
- RF kernel is doubly-stochastic by averaging 1/leaf-size co-membership over trees.
- Diffusion maps on the RF kernel give the encoder; Nyström handles out-of-sample.
- k-NN decoder (weighted average + categorical vote) is the fast/accurate default.
- For large `n`, the dense kernel is O(n^2) memory; sub-sample or use approximate eigensolvers.
